In [ ]:
import pandas as pd
import numpy as np
import datetime
from IPython.display import display

## importing the raw data and creating tables that we needed 

drive                  = pd.read_csv("~/Downloads/analyst-challenge/analyst-challenge-data/drive.csv", header = 0)
vehicle                = pd.read_csv("~/Downloads/analyst-challenge/analyst-challenge-data/vehicle.csv", header = 0)
vehicle.rename(columns = {'Model':'model'}, inplace = True)
drive_vehicle          = pd.merge(drive[["datetime","vehicle_spec_id","trip_id","velocity"]], 
                         vehicle[["vehicle_spec_id","make","model"]], 
                         how ="left", on = ["vehicle_spec_id"])
make_model             = drive_vehicle[["trip_id","make","model"]].drop_duplicates()
make_model_indexed     = make_model.set_index('trip_id')

display(make_model.isnull().sum().sort_values(ascending = False))
display(drive_vehicle.loc[drive_vehicle['make'].isnull() == False].trip_id.nunique())
## only 194 trips could join to the raw vehicle data with vehicle_spec_id and 119 are NaN that means 75 trips with not null


## converting to PST time and creating date_pst column
drive.set_index(['datetime','trip_id'])
drive['date_pst']     = pd.to_datetime(drive['datetime'], format = '%Y-%m-%d %H:%M:%S')
drive['date_pst'].dt.tz_localize('utc').dt.tz_convert('US/Central')
drive['date_pst']     = pd.to_datetime(drive['date_pst']).dt.date
#display(drive)


display(drive[drive['trip_id'] == "00922df3be5a4589ab385d0c2da2dd81"])

## assuming every trip was one second long and creating aggregate table called daily_trip grouped by PST date then trip_id


daily_trip = drive.groupby(['trip_id','date_pst']).agg(start            = pd.NamedAgg(column="datetime", aggfunc ="min"),
                                                       end              = pd.NamedAgg(column="datetime", aggfunc ="max"),
                                                       trip_count       = pd.NamedAgg(column="datetime", aggfunc ="count"),
                                                       average_velocity = pd.NamedAgg(column ="velocity", aggfunc="mean"))


daily_trip['start'] =  pd.to_datetime(daily_trip['start'],  format ='%Y-%m-%d %H:%M:%S')
daily_trip['end']   =  pd.to_datetime(daily_trip['end'],    format ='%Y-%m-%d %H:%M:%S')
#print(Daily_trip.to_string())

daily_trip['trip_duration_minutes'] = daily_trip['end'] - Daily_trip['start']
daily_trip['trip_duration_minutes'] = daily_trip['trip_duration_minutes']/np.timedelta64(1,'m')


#daily_trip['trip_count'].sum()-daily_trip['trip_duration_minutes'].sum()*60
## This should be 202 to confirm that trips are 1 seconds long

## When calculating distance_travelled using speed instead of velocity will be more accurate, however we only have velocity in the raw data 
## so we will neglect the errors from velocity being a vector

daily_trip['distance_travelled']    = daily_trip['average_velocity']* daily_trip['trip_duration_minutes']/60
daily_trip

## Bringing in the make and model into the final result and transform the dataframe into desired output

daily_trip_resetindex        = daily_trip.reset_index(level = [0,1])
daily_trip_indexed           = daily_trip_resetindex.set_index('trip_id')
daily_trip_final             = pd.merge(daily_trip_indexed,make_model_indexed,on='trip_id')
daily_trip_final             = daily_trip_final.drop(columns = ['start', 'end','trip_count','average_velocity'])
daily_trip_final['date_pst'] =  pd.to_datetime(daily_trip_final['date_pst'], format = '%Y-%m-%d')
daily_trip_final.reset_index(inplace = True)
daily_trip_final             = daily_trip_final.set_index(['date_pst','trip_id'])
display(daily_trip_final[['make', 'model', 'trip_duration_minutes','distance_travelled']])



In [ ]:
print(vehicle.apply(lambda col: col.nunique()))

## Trip SQL

In [ ]:
import sqlite3
# install the ipython-sql libray
!pip install ipython-sql

# We will sqlite3 library and create a connection
conn = sqlite3.connect('clearbanktask.db')

vehicle.to_sql('vehicle', conn)
drive.to_sql('drive', conn)
%load_ext sql

%sql sqlite:///clearbanktask.db
 

In [ ]:
%%sql sqlite:// 
SELECT drive.trip_id, 
AVG(drive.eng_load/ 255)*100 as average_eng_load_perc, 
AVG(velocity) as average_velocity,
((Max(fuel_level)- Min(fuel_level))/255)*fuel_tank_capacity as fuel_used  

FROM drive inner join vehicle 
ON drive.vehicle_spec_id = vehicle.vehicle_spec_id 
GROUP BY drive.trip_id

%sql SELECT drive.trip_id, AVG(drive.eng_load/ 255)*100 as average_eng_load_perc,AVG(velocity) as average_velocity,((Max(fuel_level)- Min(fuel_level))/255)*fuel_tank_capacity as fuel_used FROM drive inner join vehicle on drive.vehicle_spec_id = vehicle.vehicle_spec_id group by drive.trip_id




## NOTES

In [ ]:
# 1. There are 2 rows in drive data with negative velocity

%sql SELECT velocity,count(*) from drive where velocity <= 0 group by velocity order by count(*) desc

In [ ]:
# 2. Need to investingate into 388 records with 0 velocity as it might have impact in caluclating the distance_travelled. Distanc doesn't depend on direction

%sql SELECT count(*) from drive where velocity = 0

In [ ]:
# 3. Confirming the there is 75 rows expected from the second output as non null make and model 

%sql SELECT count(distinct drive.trip_id) FROM drive inner join vehicle on drive.vehicle_spec_id = vehicle.vehicle_spec_id

In [ ]:

%sql SELECT name FROM sqlite_master WHERE type='table'